In [ ]:
import functools
from multiprocessing import Pool
import os

import numpy as np
import scipy.optimize as optimize
from scipy import integrate 
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rc('xtick', labelsize="large") 
matplotlib.rc('ytick', labelsize="large") 
%matplotlib inline

import experiment

In [ ]:
# load data
data2011 = np.loadtxt("data/emission/gt_2011.csv",skiprows=1,delimiter=',')
data2012 = np.loadtxt("data/emission/gt_2012.csv",skiprows=1,delimiter=',')
data2013 = np.loadtxt("data/emission/gt_2013.csv",skiprows=1,delimiter=',')
data2014 = np.loadtxt("data/emission/gt_2014.csv",skiprows=1,delimiter=',')
data2015 = np.loadtxt("data/emission/gt_2015.csv",skiprows=1,delimiter=',')
data = np.concatenate((data2011,data2012,data2013,data2014,data2015), axis=0)
y = data[:, -1]
x = data[:, :-2]

In [ ]:
# set constants
(n, d) = np.shape(x)
cuy = 110
cly = 40
sigma = 1.
alpha = 0.3

cux = np.array([10., 1030., 100., 6., 30., 1100., 570., 170., 15.])
cux_n = np.repeat(cux, n_dash, axis=0).reshape((d, n_dash)).transpose()
clx = np.array([ 5., 1000.,  70., 4., 20., 1000., 530., 130., 10.])
clx_n = np.repeat(clx, n_dash, axis=0).reshape((d, n_dash)).transpose()

In [ ]:
if __name__ ==  '__main__': 
    num_processors = os.cpu_count()
    epsilons = np.array([5., 10., 25])
    num_trials = 1000
    ns = np.array([100, 1000, 10000])

    his = np.zeros((len(epsilons), len(ns), num_trials, d))
    
    for j, epsilon in enumerate(epsilons):
        expe = functools.partial(experiment.experiment_private, x=x, y=y, 
                                sigma=sigma, alpha=alpha, epsilon=epsilon, cuy=cuy, cly=cly,
                                cux=cux, clx=clx)
        for k, n_dash in enumerate(ns):
            with Pool(processes = num_processors) as p:
                output = np.array(p.map(expe,np.ones(num_trials,dtype=int)*ns[k]))
            his[j, k] = output
    
    # np.save("outputs/appE1_his.npy", his)

In [ ]:
epsilons = np.array([5., 10., 25])
ns = np.array([100, 1000, 10000])

his = np.load("outputs/appE1_his.npy")
covs = np.zeros((len(epsilons), len(ns)))
for j in range(len(epsilons)):
    for k in range(len(ns)):
        covs[j, k] = np.linalg.norm(np.cov((his[j, k]-np.average(his[j,k], axis=0)).transpose()))

In [ ]:
plot = plt.semilogy
plot(ns, covs[0], "-x", label="eps=5.0")
plot(ns, covs[1], "--o", label="eps=10")
plot(ns, covs[2], "-s", label="eps=25")
plot(ns, 500/ns, "--", label="O(1/n)")
plt.xlabel("n",fontsize="large")
plt.ylabel("Frobenius norm",fontsize="large")
plt.legend(fontsize="large")
# plt.savefig("appE1.eps")
plt.show()